# Artificial Neural Network

## Data preprocessing

### Importing the libraries

In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [38]:
tf.__version__

'2.15.0'

### Loading dataset

In [40]:
data = pd.read_csv('Churn_Modelling.csv')
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

### Encoding categorical data

In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

###One Hot Encoding the "Geography" column

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Spliting data into the train and test set

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Building the ANN

#### Initializing ANN

In [49]:
ann = tf.keras.models.Sequential()

#### Adding the input layer and the first hidden layer

In [50]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

#### Adding the second hidden layer

In [51]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

#### Adding the output layer

In [52]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

### Training the ANN

#### Compiling the ANN

In [53]:
ann.compile(optimizer='adam',loss='binary_crossentropy' ,metrics=['accuracy'])

#### Training the ANN on the training set

In [54]:
ann.fit(X_train, y_train, batch_size = 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6389 - accuracy: 0.6404
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4819 - accuracy: 0.7968
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4461 - accuracy: 0.7979
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4213 - accuracy: 0.8051
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4015 - accuracy: 0.8210
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3879 - accuracy: 0.8331
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3786 - accuracy: 0.8374
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3704 - accuracy: 0.8419
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3645 - accuracy: 0.8436
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3598 - accura

### Making the predictions and evaluating the model

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer ?

In [57]:
print(ann.predict(scaler.transform([[1 ,0 ,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 181ms/step
[[False]]




Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.


### Predicting the test results

In [58]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1515   80]
 [ 200  205]]


0.86